<a href="https://colab.research.google.com/github/alessandrow19/-continuos-integration/blob/main/dio_desafio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def fetch_and_clean_text(url)->str:
  try:
      response = requests.get(url)
      response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
      soup = BeautifulSoup(response.content, 'html.parser')
      text = soup.get_text()

      # Remove excessive whitespace (spaces, tabs, newlines)
      cleaned_text = re.sub(r'\s+', ' ', text).strip()

      # You might need more specific cleaning based on the actual output,
      # for example, removing header/footer text that remains after get_text()
      # print(cleaned_text)
      # For a more refined approach, you could navigate the soup tree to find the main article content

      # A more robust approach would be to find the main article container and extract text from there
      # Let's try to find potential article containers
      article_container = soup.find('div', class_='c-article-base') or soup.find('article')
      if article_container:
          article_text = article_container.get_text()
          # Clean the article text
          cleaned_article_text = re.sub(r'\s+', ' ', article_text).strip()
          return cleaned_article_text
      else:
          # Fallback to the previous cleaning if a specific article container is not found
          return cleaned_text


  except requests.exceptions.RequestException as e:
      print(f"Error fetching the URL: {e}")
  except Exception as e:
      print(f"An error occurred: {e}")




In [ ]:
import os
from openai import AzureOpenAI
from google.colab import userdata

endpoint = "https://dio-azure-desafio.openai.azure.com/"
model_name = "o4-mini"
deployment = "o4-mini"

# Get the subscription key from Colab's Secrets Manager
subscription_key = userdata.get('AZURE_OPENAI_SUBSCRIPTION_KEY')
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)


def generate_response(text, lang):

   messages=[
        {
            "role": "system",
            "content": "Você é um especilista em traduzir textos ",
        },
        {
            "role": "user",
            "content": f"traduza o {text} para {lang} e responda em formato markdown",
        }
    ]


   response = client.chat.completions.create(
          messages=messages,
          max_completion_tokens=10000,
          model=deployment
   )

   print(response.choices[0].message.content)

In [ ]:
url=userdata.get('URL')


texto=fetch_and_clean_text(url)
generate_response(texto, "pt-br")